In [34]:
import os
import datetime
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
import matplotlib.patches as mpatches
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cbook as cbook
import netCDF4
import xarray as xr
import pandas as pd
import glob
from typing import List
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.simplefilter('ignore')
from pathlib import Path
import seaborn as sns
from numpy.ma import log10
import itertools
from matplotlib.pyplot import subplots, colorbar, cm
import matplotlib.colors as mcolors
from numpy.ma import log10
from matplotlib.dates import date2num, DateFormatter, HourLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [35]:
# This is the folder where the original data is stored
original_data_path = Path("/data/home/levin/data/datastream/nsa/nsathermocldphaseC1.c1/") ##EDIT!

# This is the folder where you'll save the new data
hand_labeled_data_path = Path("/home/charris/HandLabeled")##EDIT!

# This is the folder where you'll save the plots
# plots_path = Path("/home/goldberger/CLDPHS_VAP_Aircraft/Plots/")##EDIT!

In [36]:
year = '2023'##EDIT!
month = '01'##EDIT!

filelist = glob.glob(os.path.join(original_data_path, '*' + year + month + '*.nc'))
filelist.sort()
date_str_list = [filelist[mj][len(filelist[mj])-18:len(filelist[mj])-10] for mj in range(len(filelist))]
print(date_str_list)

[]


In [37]:
import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.colors import ListedColormap
from numpy.typing import NDArray


def plot_cloud_phase(ds: xr.Dataset, figsize: tuple[int, int] | None = None):
    if figsize is None:
        figsize = (12, 6)

    date = pd.Timestamp(ds.time.data[0]).strftime("%Y-%m-%d")
    site = ds.attrs["site_id"]
    facility = ds.attrs["facility_id"]

    ds_5km: xr.Dataset = ds.where(ds.height < 5, drop=True)
    cloud_phase: Any = ds_5km["cloud_phase_mplgr"]

    flag_meanings = cloud_phase.attrs["flag_meanings"].split(" ")
    flag_colors = ["w", "g", "b", "r", "c", "orange", "y", "black", "dimgray"]
    cmap = ListedColormap(flag_colors)

    fig, ax = plt.subplots(
        nrows=1, sharex=True, figsize=figsize, constrained_layout=True
    )
    fig.suptitle(f"MPLGR Cloud Phase at {site} {facility} on {date}")

    plot = cloud_phase.plot(
        ax=ax,
        x="time",
        vmin=0,
        vmax=len(flag_colors),
        cmap=cmap,
        add_colorbar=False,
    )

    cbar = plt.colorbar(plot, ax=ax, pad=0.01)
    cbar.ax.set_ylabel("")
    cbar.ax.locator_params(nbins=len(flag_colors))
    cbar.set_ticks([0.5 + i for i in range(len(flag_colors))])
    cbar.set_ticklabels(flag_meanings)
    cbar.ax.minorticks_off()

    # Add contours
    if not np.isnan(ds_5km.temp.values).all():
        contours = ds_5km.temp.plot.contour(
            ax=ax,
            x="time",
            vmin=-40,
            vmax=40,
            levels=17,
            colors=["black"],
            linestyles="--",
            linewidths=0.8,
        )
        ax.clabel(contours, fontsize=8)

    ax.grid()
    format_time_xticks(ax)
    for label in ax.get_xticklabels():
        label.set_rotation(0)
        label.set_horizontalalignment("center")

    ax.set_xlabel("Time (UTC)")
    ax.set_ylabel("Height AGL (km)")

    temp_label = mlines.Line2D([], [], color="black", linestyle="--", linewidth=0.8)
    ax.legend([temp_label], ["Temperature (degC)"], loc="upper right")
    return fig

In [38]:
def format_time_xticks(
    ax: plt.Axes,
    start: int = 4,
    stop: int = 21,
    step: int = 4,
    date_format: str = "%H:%M",
):
    """----------------------------------------------------------------------------
    Formats the ticks on the x-axis of the provided `plt.Axes` nicely. Requires the
    provided `plt.Axis` to already have a plot attached to it and for the x-axis of
    the plotted data to be a datetime object (numpy / pandas / xarray OK). Sets
    major tick locations by hour according to the provided `start`, `stop`, and
    `step` parameters, and labels ticks according to the provided `date_format`.
    Has nice defaults for a plot spanning a 24-hour period.
    Args:
        ax (plt.Axes): The handle for the axes object on which to format the ticks.
        start (int, optional): Hour in which to start the xticks. Defaults to 4.
        stop (int, optional): Hour in which to stop the xticks. Defaults to 21.
        step (int, optional): The step in between major xticks. Defaults to 4.
        date_format (str, optional): The format to use for xtick labels. Defaults
        to "%H-%M".
    ----------------------------------------------------------------------------"""
    ax.xaxis.set_major_locator(mpl.dates.HourLocator(byhour=range(start, stop, step)))  # type: ignore
    ax.xaxis.set_major_formatter(mpl.dates.DateFormatter(date_format))  # type: ignore
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

In [39]:
# This function loads in a single file

def getCLOUDVAP(date_input, date_str_list, filelist): # "YYYYMMDD"

    # Get Day
    try: 
        date_index = date_str_list.index(date_input)
    except ValueError:
        print('Check if date_input as a string in format YYYYMMDD.')
        print('Try a date between ' + date_str_list[0] + ' - ' + date_str_list[-1])

    # Get Data
    with xr.open_dataset(filelist[date_index]) as ds:
        ds
    time = ds['time'].data
    height = ds['height'].data
    cldphase_mplgr = ds['cloud_phase_mplgr'].data
    cldphase_layer_mplgr = ds['cloud_phase_layer_mplgr'].data
    mpl_dep = ds['mpl_linear_depol_ratio'].data
    mpl_backscat = ds['mpl_backscatter'].data * \
        (ds['height'].data**2)
    mpl_backscat[mpl_backscat <= 0] = 10**-9
    ds['mpl_backscatter'].data = mpl_backscat
    ds['mpl_backscatter'].attrs[
        'units'] = f"log10({ds['mpl_backscatter'].attrs['units']})"

    arscl_cloud_top = ds['cloud_layer_top_height'].data
    arscl_cloud_base = ds['cloud_layer_base_height'].data
    arscl_ze = ds['reflectivity_best_estimate'].data
    arscl_ze_snr = ds['radar_signal_to_noise_ratio'].data
    arscl_w = ds['spectral_width'].data
    arscl_mdv = ds['mean_doppler_velocity'].data
    combined_ze = arscl_ze.copy()
    combined_mdv = arscl_mdv.copy()
    combined_w = arscl_w.copy()
    mwr_lwp = ds['mwrret1liljclou_be_lwp'].data
    sonde_temp = ds['temp'].data

    mpl_backscat_ts = log10(mpl_backscat).transpose()

    cldphase_layer_pixel = np.zeros(mpl_backscat.shape, dtype=int)
    for j, k in itertools.product(range(0, len(time)), range(0, 10)):
        cloud_top_jk = arscl_cloud_top[j, k] * 0.001
        cloud_base_jk = arscl_cloud_base[j, k] * 0.001
        if(cloud_top_jk > 0) and (cloud_base_jk > 0):
            if(cldphase_layer_mplgr[j, k] == 1):
                cldphase_layer_pixel[j, np.where(
                    (height <= cloud_top_jk) & (height >= cloud_base_jk))] = 1
            if(cldphase_layer_mplgr[j, k] == 2):
                cldphase_layer_pixel[j, np.where(
                    (height <= cloud_top_jk) & (height >= cloud_base_jk))] = 2
            if(cldphase_layer_mplgr[j, k] == 3):
                cldphase_layer_pixel[j, np.where(
                    (height <= cloud_top_jk) & (height >= cloud_base_jk))] = 3

    return {"time": time, "height": height, "cldphase_mplgr": cldphase_mplgr,"cldphase_layer_mplgr": cldphase_layer_mplgr, "mpl_dep": mpl_dep, "mpl_backscat": mpl_backscat,"arscl_cloud_top": arscl_cloud_top, "arscl_cloud_base": arscl_cloud_base, "arscl_ze": arscl_ze, "arscl_ze_snr": arscl_ze_snr, "arscl_w": arscl_w, "arscl_mdv": arscl_mdv,"combined_ze": combined_ze, "combined_mdv": combined_mdv, "combined_w": combined_w,"mwr_lwp": mwr_lwp, "sonde_temp": sonde_temp, "mpl_backscat_ts": mpl_backscat_ts,"cldphase_layer_pixel": cldphase_layer_pixel}
# How to use:
# d = getCLOUDVAP('YYYYMMDD', date_str_list, filelist)
# time = d["time"]

In [40]:
day = '14' ##EDIT!
date_input = year + month + day
print('This is the specific day: ' + date_input)

d = getCLOUDVAP(date_input, date_str_list, filelist)

This is the specific day: 20230114
Check if date_input as a string in format YYYYMMDD.


IndexError: list index out of range

In [ ]:
import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.colors import ListedColormap
from numpy.typing import NDArray


def plot_cloud_phase(ds: xr.Dataset, figsize: tuple[int, int] | None = None):
    if figsize is None:
        figsize = (12, 6)

    date = pd.Timestamp(ds.time.data[0]).strftime("%Y-%m-%d")
    site = ds.attrs["site_id"]
    facility = ds.attrs["facility_id"]

    ds_5km: xr.Dataset = ds.where(ds.height < 5, drop=True)
    cloud_phase: Any = ds_5km["cloud_phase_mplgr"]

    flag_meanings = cloud_phase.attrs["flag_meanings"].split(" ")
    flag_colors = ["w", "g", "b", "r", "c", "orange", "y", "black", "dimgray"]
    cmap = ListedColormap(flag_colors)

    fig, ax = plt.subplots(
        nrows=1, sharex=True, figsize=figsize, constrained_layout=True
    )
    fig.suptitle(f"MPLGR Cloud Phase at {site} {facility} on {date}")

    plot = cloud_phase.plot(
        ax=ax,
        x="time",
        vmin=0,
        vmax=len(flag_colors),
        cmap=cmap,
        add_colorbar=False,
    )

    cbar = plt.colorbar(plot, ax=ax, pad=0.01)
    cbar.ax.set_ylabel("")
    cbar.ax.locator_params(nbins=len(flag_colors))
    cbar.set_ticks([0.5 + i for i in range(len(flag_colors))])
    cbar.set_ticklabels(flag_meanings)
    cbar.ax.minorticks_off()

    # Add contours
    if not np.isnan(ds_5km.temp.values).all():
        contours = ds_5km.temp.plot.contour(
            ax=ax,
            x="time",
            vmin=-40,
            vmax=40,
            levels=17,
            colors=["black"],
            linestyles="--",
            linewidths=0.8,
        )
        ax.clabel(contours, fontsize=8)

    ax.grid()
    format_time_xticks(ax)
    for label in ax.get_xticklabels():
        label.set_rotation(0)
        label.set_horizontalalignment("center")

    ax.set_xlabel("Time (UTC)")
    ax.set_ylabel("Height AGL (km)")

    temp_label = mlines.Line2D([], [], color="black", linestyle="--", linewidth=0.8)
    ax.legend([temp_label], ["Temperature (degC)"], loc="upper right")
    return fig

In [ ]:
def format_time_xticks(
    ax: plt.Axes,
    start: int = 4,
    stop: int = 21,
    step: int = 4,
    date_format: str = "%H:%M",
):
    """----------------------------------------------------------------------------
    Formats the ticks on the x-axis of the provided `plt.Axes` nicely. Requires the
    provided `plt.Axis` to already have a plot attached to it and for the x-axis of
    the plotted data to be a datetime object (numpy / pandas / xarray OK). Sets
    major tick locations by hour according to the provided `start`, `stop`, and
    `step` parameters, and labels ticks according to the provided `date_format`.
    Has nice defaults for a plot spanning a 24-hour period.
    Args:
        ax (plt.Axes): The handle for the axes object on which to format the ticks.
        start (int, optional): Hour in which to start the xticks. Defaults to 4.
        stop (int, optional): Hour in which to stop the xticks. Defaults to 21.
        step (int, optional): The step in between major xticks. Defaults to 4.
        date_format (str, optional): The format to use for xtick labels. Defaults
        to "%H-%M".
    ----------------------------------------------------------------------------"""
    ax.xaxis.set_major_locator(mpl.dates.HourLocator(byhour=range(start, stop, step)))  # type: ignore
    ax.xaxis.set_major_formatter(mpl.dates.DateFormatter(date_format))  # type: ignore
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)

In [ ]:
# This function makes plots pretty :)

def set_axes(ax, font_size, font_weight, up_hgt, x_nvis,color_bar_label, padding):
    ax.set_xlabel(ax.get_xlabel(), fontsize=font_size, fontweight=font_weight)
    ax.set_ylabel(ax.get_ylabel(), fontsize=font_size, fontweight=font_weight)
    ax.set_title(ax.get_title(), fontsize=font_size, fontweight=font_weight)
    
    for tick in ax.get_xticklabels():
        tick.set_fontsize(font_size)
    
    for tick in ax.get_yticklabels():
        tick.set_fontsize(font_size)
        
    clrs = sns.color_palette("colorblind", 9)
    font = {'weight': font_weight, 'size': font_size}
    matplotlib.rc('font', **font)
    
    ax.set_ylim(0, up_hgt)
    plt.locator_params(axis='y', nbins=5)
    ax.set_ylabel('Height (km)')
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", "3%", pad=padding)
    cb = colorbar(pplot, cax=cax)
    cb.ax.set_ylabel((color_bar_label))
    cb.ax.set_ylabel(cb.ax.get_ylabel(), fontsize=font_size, fontweight=font_weight)
    
    if x_nvis ==1:
        plt.setp(ax.get_xticklabels(), visible=False)
        
    return cb, ax
# # Example usage:
# fig, ax = plt.subplots()
# ax.plot([1, 2, 3, 4], [2, 4, 1, 3])

# # Call the function to set the font size for the axes
# set_axes(ax, 12, 'bold', 8, 1)

# plt.show()

In [ ]:
day = '14' ##EDIT!
date_input = year + month + day
print('This is the specific day: ' + date_input)

d = getCLOUDVAP(date_input, date_str_list, filelist)

In [ ]:
# Plot Radar and MPL Data
color_map = cm.get_cmap("jet")
# mpl backscatter
fig, (ax) = plt.subplots(1,1,figsize=(14,1), sharex=True, sharey=True)
pplot = ax.pcolormesh(d['time'], d['height'], d['mpl_backscat_ts'], vmin=-3, vmax=2, cmap=color_map, shading='auto')
set_axes(ax, 12, 'bold', 8, 1, r'($\beta$)',"12%")
# mpl_linear_depol_ratio
fig, (ax1) = plt.subplots(1,1,figsize=(14,1), sharex=True, sharey=True)
pplot = ax1.pcolormesh(d['time'], d['height'], d['mpl_dep'].transpose(),vmin=0, vmax=0.6, cmap=color_map, shading='auto')
set_axes(ax1, 12, 'bold', 8, 1,r'MPL $\beta$',"12%")
# radar Ze
fig, (ax2) = plt.subplots(1,1,figsize=(14,1), sharex=True, sharey=True)
pplot = ax2.pcolormesh(d['time'], d['height'], d['combined_ze'].transpose(),vmin=-40, vmax=20, cmap=color_map, shading='auto')
set_axes(ax2, 12, 'bold', 8, 1,'Ze (dBZ)',"12%")
# plot radar Doppler velocity
fig, (ax3) = plt.subplots(1,1,figsize=(14,1), sharex=True, sharey=True)
pplot = ax3.pcolormesh(d['time'], d['height'], d['combined_mdv'].transpose(),vmin=-1, vmax=1, cmap=color_map, shading='auto')
set_axes(ax3, 12, 'bold', 8, 1,'MDV (m/s)',"12%")
# plot radar Doppler spectra width
fig, (ax4) = plt.subplots(1,1,figsize=(14,1), sharex=True, sharey=True)
pplot = ax4.pcolormesh(d['time'], d['height'], d['combined_w'].transpose(),vmin=0, vmax=0.5, cmap=color_map, shading='auto')
set_axes(ax4, 12, 'bold', 8, 1,'W (m/s)',"12%")

################################

# Plot cloud phase datasets from ground

fig, (ax) = plt.subplots(1,1,figsize=(14,2), sharex=True, sharey=True)

# plot cldphase_pixel
cmap = ListedColormap(['w', 'g', 'b', 'r','gray','m','y','brown','c'])
pplot = ax.contour(d['time'], d['height'], d['sonde_temp'].transpose(),vmin=-40, vmax=40, levels = 9, colors = 'black', linestyles = '--',linewidths = 0.5)
ax.clabel(pplot,fontsize = 4,inline = True,fmt='%1.0f')
pplot = ax.pcolormesh(d['time'], d['height'], d['cldphase_mplgr'].transpose(),vmin=0, vmax=9, cmap=cmap, shading='auto')

cb, ax = set_axes(ax, 12, 'bold', 8, 0,'',"4%")

cb.ax.locator_params(nbins=8)
#cb.set_ticks([0.33, 1.33, 2.33, 3.33, 4.33, 5.33, 6.33, 7.33, 8.33])
cb.set_ticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
cb.set_ticklabels(['Clear', 'liquid', 'Ice', 'Mixed','Drizzle','Liquid + Drizzle','Rain','Snow','Unknown'])

date_fmt = DateFormatter('%H:00')
hour_loc = HourLocator(interval=5)
ax.xaxis.set_major_formatter(date_fmt)
ax.xaxis.set_major_locator(hour_loc)


In [ ]:
# Open in xarray format for easier indexing

date_index = date_str_list.index(date_input)
with xr.open_dataset(filelist[date_index]) as ds:
        ds

In [ ]:
plot_cloud_phase(ds)